In [1]:

import gym
import gym_joinemio
from gym_joinemio.envs.player import RandomPlayer
import torch
import random
from collections import defaultdict, deque
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
import copy
from pprint import pprint
import math
from tqdm.autonotebook import tqdm
from torch.utils.data import DataLoader
from torchvision import transforms
from joblib import Parallel, delayed
import gc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
env = gym.make('joinemio-v0')
player1 = RandomPlayer()
player2 = RandomPlayer()

iterations = 1
for i in range(iterations): # Iterate the game
    one_game = env.play_one_game(player1, player2, each_step_render=False)

or e in env.recording: print(e)

SyntaxError: invalid syntax (<ipython-input-2-c79b359dbc9a>, line 9)

In [49]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# Try N+9:

In [4]:
def trainNNet(examples,optimizer,nnet):
    nnet = NNET(nnet,device=device, optimizer=optimizer)
    nnet.train(examples)
    return nnet.nn


class CNN(nn.Module):
    """
    input: 6 x 7 x 1 (reward) x ...?
    output: 7 (best possible move?)
    """
    def __init__(self,action_size=7,hidden_size=128):
        super(Connect4Net, self).__init__()

    def forward(self, s):
        pass

class SampleDataset:
    def __init__(self,samples):
        self.samples = samples
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self,index):
        pass
        return torch.tensor(object)
    
    
class NNET:
    """
    Helper class to train and predict
    """
    def __init__(self,nnet,device,optimizer,epoch=10):
        self.nn = nnet
        self.epoch = epoch
        self.device = device
        self.optimizer = optimizer
    
    def train(self,examples):
        optimizer = self.optimizer
        criterion = self.criterion
        model = self.nn
        device = self.device
        
        model.to(device)
        model.train()
        train_loader = get_trainloader(examples)
        
        for e in range(self.epoch):
            total_loss = 0
            for d in train_loader:
                optimizer.zero_grad()

